I'm a beginner, so please be gentle with me.

In [16]:
import os
os.chdir('C:/Users/Takanori/Desktop/Kaggle/TPS2205/input')

In [17]:
# %%capture
# pip install pytorch-tabnet

# DataSet & Library Loading

In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time, gc

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

In [20]:
class CFG:
    batch_size = 100_000
    epochs = 10_000  # 最大のepoch数
    folds = 10  # クロスバリデーションにおけるfold数
    seed = 42  # TODO 機能させる
    target = 'target'  # 目標変数
    model_path = "models"  # モデルを保存するフォルダ
    test_pred = []  # 各foldで作ったモデル別の、testを予測した結果のカラム名
    pred = 'pred'  # trainを予測した結果のカラム名
    increase_batch = 9999  # 何epoch間lossが改善しなかったbatchを増やすか
    early_stopping = 100  # 何epochでearly stoppingをするか
    lr = 1  # optimizerの学習率
    # min_lr = 1e-3  # 学習率の最低値
    lr_patience = 10  # 学習率を引き下げる間隔
    lr_factor = 0.5  # lrを何倍にするか
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    delta = 1e-4  # AUC等がいくつ変わらなかったら学習などをやめるか

In [21]:
os.makedirs(CFG.model_path, exist_ok=True)

## FE

In [22]:
# データセット読み込み
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
df_sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
display(df_train.head())
display(df_test.head())
display(df_sub.head())

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30
0,900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,...,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0
1,900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,...,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0
2,900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,...,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1
3,900003,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,...,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0
4,900004,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,...,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2


,id,target
0,900000,0.5
1,900001,0.5
2,900002,0.5
3,900003,0.5
4,900004,0.5


In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
display(df_train.shape)
display(df_test.shape)

(900000, 33)

(700000, 32)

In [25]:
df_all = pd.concat([df_train, df_test])

In [26]:
df_all['i_02_21'] = (df_all.f_21 + df_all.f_02 > 5.2).astype(int) - (df_all.f_21 + df_all.f_02 < -5.3).astype(int)
df_all['i_05_22'] = (df_all.f_22 + df_all.f_05 > 5.1).astype(int) - (df_all.f_22 + df_all.f_05 < -5.4).astype(int)
i_00_01_26 = df_all.f_00 + df_all.f_01 + df_all.f_26
df_all['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

In [27]:
# 不要データを削除
df_all.drop(['id'], axis=1, inplace=True)

In [28]:
# NA埋め
df_all.fillna(df_all.mean(numeric_only=True), inplace=True)

In [29]:
# f_27を分解
def splitter(text):
    arr = tuple(text)
    return arr

column_list = ['f_27_' + str(i) for i in range(10)]
df_all[column_list] = df_all.apply(lambda x: splitter(x['f_27']), axis=1, result_type='expand')

In [30]:
# f_27_0~9を数値に
for column in column_list:
    df_all[column] = df_all[column].apply(lambda x: ord(x)-ord('A'))

In [31]:
# f_27に出てくる文字が、何種類あるかを返す
def calc_cnt_of(txt):
    from collections import Counter
    cnt_of = Counter(list(txt))
    return len(cnt_of)

df_all['len_cnt_of'] = df_all['f_27'].apply(calc_cnt_of)

In [32]:
# 不要になったカテゴリ変数を削除
df_all.drop(['f_27'], axis=1, inplace=True)

In [33]:
df_train = df_all[:len(df_train)]
df_test = df_all[len(df_train):]

In [34]:
df_train.to_pickle('train_fe.pkl')
df_test.to_pickle('test_fe.pkl')

## Pytorch_Tabular

In [35]:
df_train = pd.read_pickle('train_fe.pkl')
df_test = pd.read_pickle('test_fe.pkl')

## Define Pytorch Model

# Pytorch Loss Function

In [36]:
criterion = nn.BCEWithLogitsLoss()

# Pytorch Training

In [ ]:
class LossChecker:
    # 指定した回数、loss等が改善されていなければTrueを返す
    def __init__(self, patience=20, strategy="min", delta=1e-6):
        self.patience = patience
        self.bef_epoch = 0
        self.strategy = strategy
        self.delta = delta
        if strategy == "max":
            self.val = -float('inf')
        else:
            self.val = float('inf')

    def step(self, epoch, loss):
        if self.strategy == "max":
            if self.val < loss:
                self.val = loss + self.delta
                self.bef_epoch = epoch
            if epoch - self.bef_epoch > self.patience:
                self.bef_epoch = epoch
                return True
            else:
                return False
        else:
            if self.val > loss:
                self.val = loss - self.delta
                self.bef_epoch = epoch
            if epoch - self.bef_epoch > self.patience:
                self.bef_epoch = epoch
                return True
            else:
                return False


In [37]:
# 学習の対象とする特徴量を列挙する
all_features = df_train.columns.tolist()
all_features.remove(CFG.target)

In [38]:
# カテゴリ変数を列挙する
cat_idxs = []
cat_dims = []
cat_emb_dim = []
for i, col in enumerate(df_all.columns):
    if col == CFG.target:
        continue
    if df_all[col].dtype in ['object', 'int64']:
        cat_idxs.append(i)
        cat_dims.append(len(df_all[col].unique()))
        cat_emb_dim.append(1)

In [39]:
df_train.shape

(900000, 45)

In [40]:
for i, d in zip(cat_idxs, cat_dims):
    print(i, d)

7 17
8 16
9 16
10 16
11 15
12 17
13 14
14 14
15 15
16 16
17 15
18 14
28 2
29 3
34 2
35 15
36 2
37 15
38 15
39 2
40 15
41 20
42 15
43 15
44 9


In [16]:
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier

In [17]:
CFG.batch_size

100000

In [18]:
df_train = df_train.sample(frac=1).reset_index(drop=True)  # df_trainをシャッフル

skf = StratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)

for fold, (train_index, valid_index) in enumerate(skf.split(df_train, df_train[CFG.target])):
    print('-----', fold, '-----')

    X_train = df_train[all_features].to_numpy()[train_index]
    y_train = df_train[CFG.target].to_numpy()[train_index]
    X_valid = df_train[all_features].to_numpy()[valid_index]
    y_valid = df_train[CFG.target].to_numpy()[valid_index]
    X_test = df_test[all_features].to_numpy()

    # fold毎に初期化する設定
    # TODO: TABNETを実装
    # https://www.kaggle.com/code/hiro5299834/tps-may-2022-tabnet-baseline
    # https://github.com/dreamquark-ai/tabnet
    tabnet_params = dict(
        n_d=8,  # Width of the decision prediction layer. Bigger values gives more capacity to the model with the risk of overfitting. Values typically range from 8 to 64.
        n_a=8,  # Width of the attention embedding for each mask. According to the paper n_d=n_a is usually a good choice. (default=8)
        n_steps=5,  # Number of steps in the architecture (usually between 3 and 10)
        gamma=1.3,  # This is the coefficient for feature reusage in the masks. A value close to 1 will make mask selection least correlated between layers. Values range from 1.0 to 2.0.
        # cat_idxs=cat_idxs, # list of int (default=[] - Mandatory for embeddings). List of categorical features indices.
        # cat_dims=cat_dims,  # list of int (default=[] - Mandatory for embeddings). List of categorical features number of modalities (number of unique values for a categorical feature) /!\ no new modalities can be predicted
        # cat_emb_dim=cat_emb_dim,  # list of int (optional). List of embeddings size for each categorical features. (default =1)
        n_independent=3,  # Number of independent Gated Linear Units layers at each step. Usual values range from 1 to 5.
        n_shared=3,  # Number of shared Gated Linear Units at each step Usual values range from 1 to 5.
        # epsilon=1e-15   # Should be left untouched.
        seed=CFG.seed,
        momentum=0.01,  # Momentum for batch normalization, typically ranges from 0.01 to 0.4 (default=0.02)
        clip_value=None,  # float (default None). If a float is given this will clip the gradient at clip_value.
        lambda_sparse=1e-6,  # float (default = 1e-3). This is the extra sparsity loss coefficient as proposed in the original paper. The bigger this coefficient is, the sparser your model will be in terms of feature selection. Depending on the difficulty of your problem, reducing this value could help.

        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(
            lr=1e-2,
            weight_decay=1e-7
        ),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        scheduler_params=dict(
            mode='max',
            factor=0.9,
            patience=3,
            min_lr=1e-6,
        ),
        verbose=10,  # Verbosity for notebooks plots, set to 1 to see every epoch, 0 to get None.
        device_name='auto',  #  str (default='auto') 'cpu' for cpu training, 'gpu' for gpu training, 'auto' to automatically detect gpu
        mask_type='sparsemax',  # (default='sparsemax') Either "sparsemax" or "entmax" : this is the masking function to use for selecting features.
    )

    # Defining TabNet model
    model = TabNetClassifier(**tabnet_params)
    model.fit(
        X_train=X_train,
        y_train=y_train,
        from_unsupervised=None,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=["train", "valid"],
        eval_metric=["auc"],
        # batch_size=CFG.batch_size,
        batch_size=32_768,
        virtual_batch_size=4_096,
        max_epochs=200,
        drop_last=True,
        pin_memory=True,
        patience=20,
        num_workers=4,
    )
    df_train.loc[valid_index, CFG.pred] = model.predict_proba(X_valid)[:, -1]
    auc = roc_auc_score(y_true=y_valid, y_score=df_train.loc[valid_index, CFG.pred])
    print('AUC:', auc)
    df_test[CFG.pred + str(fold)] = model.predict_proba(X_test)[:, -1]

    # TEST
    print('----- this is test run -----')
    break

----- 0 -----
Device used : cuda
epoch 0  | loss: 0.74134 | train_auc: 0.55016 | valid_auc: 0.54774 |  0:01:20s


: 

: 

# predictions

In [ ]:
cols = [col for col in df_test.columns if CFG.tab_pred in col]

df_sub[CFG.target] = df_test[cols].mean(axis=1)
df_sub.to_csv("submission.csv", index=False)
df_sub

In [ ]:
# df_sub[CFG.target] = (df_sub[CFG.target] > 0.5).astype(int)

In [ ]:
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub.head(10)

,id,target
0,900000,0.999988
1,900001,0.999955
2,900002,0.000021
3,900003,0.000066
4,900004,0.998805
5,900005,0.005635
6,900006,0.089053
7,900007,0.999996
8,900008,0.004963
9,900009,0.999887
